# Project 2: User Assessments Report

## PART 1: Introduction
### **Business Case**
- A highly reputable tech firm offers a state of the training & assessment platform enabeling the new radical shift from a highly scaleable and highly modular-digital platform. Many publishers have taken notice and have requested to utilize the platform assuming we can demonstrate a secure, reliable, and robust data engineering pipeline that's easily quireyable for their data sceintists. 

### **Goal:** 
Develop and demonstrate a basic data pipeline to transmit and distribute event messages into kaufka and analyzed using Spark

### Data


### PART II: The Process Steps
In this section, I have documented all the major steps in setting up the data pipeline all from the command line terminal.   
I have annotated each step with the following:    
--1. A brief description of the step  
--2. The code used to execute in the cli  
--3 The output copied from the command line terminal

**Step 1: Initial Set up**  

- Make directory
- Create docker-compose file. We will be using the same file from the week 8 async. 

In [ ]:
#Make directory and 
mkdir w205/project-2-dtrinidad002

In [ ]:
# copy the yml file from course contents
cp ~/w205/course-content//08-Querying-Data/docker-compose.yml .

# read docker-compose.yml 
cat docker-compose.yml

**Step 2: Get Data File**  
-code for getting data below, followed by the output from the console  
-check the data file and verify it's not empty

In [ ]:
curl -L -o assessment-attempts-20180128-121051-nested.json https://goo.gl/ME6hjp`

**Step 3: Spin Up Docker Container Cluster**    
We will be using Kafka, Spark, Zookeeper, Cloudera and Mids containers

In [ ]:
#initiate cluster container
docker-compose up -d

In [ ]:
#verify containers are running 
docker-compose ps

**Step 4: Set up Kaufka logs**  
-We will run this in a separate terminal window  
-Snippet of the logs running on 2nd terminal

In [ ]:
jupyter@python-20210907-215615:~/w205/project-2-dtrinidad002$ docker-compose logs -f kafka
Attaching to project-2-dtrinidad002_kafka_1
kafka_1      | ===> User
kafka_1      | uid=1000(appuser) gid=1000(appuser) groups=1000(appuser)
kafka_1      | ===> Configuring ...
kafka_1      | ===> Running preflight checks ... 
kafka_1      | ===> Check if /var/lib/kafka/data is writable ...
kafka_1      | ===> Check if Zookeeper is healthy ...
kafka_1      | SLF4J: Class path contains multiple SLF4J bindings.
kafka_1      | SLF4J: Found binding in [jar:file:/usr/share/java/cp-base-new/slf4j-simple-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
kafka_1      | SLF4J: Found binding in [jar:file:/usr/share/java/cp-base-new/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
kafka_1      | SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
kafka_1      | SLF4J: Actual binding is of type [org.slf4j.impl.SimpleLoggerFactory]
kafka_1      | [main] INFO org.apache.zookeeper.ZooKeeper - Client environment:zookeeper.version=3.5.9-83df9301aa5c2a5d284a9940177808c01bc35cef, built on 01/06/2021 20:03 GMT
kafka_1      | [main] INFO org.apache.zookeeper.ZooKeeper - Client environment:host.name=d0106b002e81
kafka_1      | [main] INFO org.apache.zookeeper.ZooKeeper - Client environment:java.version=11.0.12
kafka_1      | [main] INFO org.apache.zookeeper.ZooKeeper - Client environment:java.vendor=Azul Systems, Inc.
kafka_1      | [main] INFO org.apache.zookeeper.ZooKeeper - Client environment:java.home=/usr/lib/jvm/zulu11-ca

**Step 5: Check Hadoop tmp directory to verify what we want to write is not already there**

In [ ]:
#listing the inital hadoop filings
docker-compose exec cloudera hadoop fs -ls /tmp/

In [ ]:
jupyter@python-20210907-215615:~/w205/project-2-dtrinidad002$ docker-compose exec cloudera hadoop fs -ls /tmp/
Found 2 items
drwxrwxrwt   - mapred mapred              0 2018-02-06 18:27 /tmp/hadoop-yarn
drwx-wx-wx   - root   supergroup          0 2021-11-02 04:06 /tmp/hive

***NOTES: we see that upon executing the hadoop folder we see 2 items in there**
-These files are standard and will always show up. 
-We have confirmed that there are no other files in the temp folder that is not supposed to be there

**Step 6: Create Topic "assessment-attepmts"**

***why did i choose to name the topic "assessment-attempts"?***
-I called the topic assessment-attempts in order to avoid ambiguity. Since the raw JSON file contains exam/assessments and user data, I chose to name the topic as such. It's good practice to name apply descriptive names to topics

In [ ]:
# Create topic "assessment-attempts"
docker-compose exec kafka kafka-topics --create --topic assessment_attempts --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181

In [ ]:
jupyter@python-20210907-215615:~/w205/project-2-dtrinidad002$ docker-compose exec kafka kafka-topics --create --topic assessment_attempts --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181
WARNING: Due to limitations in metric names, topics with a period ('.') or underscore ('_') could collide. To avoid issues it is best to use either, but not both.
Created topic assessment_attempts.

**Step 7: Verify newly created kaufka topic "assessment-attempts"**

In [ ]:
#Describe topic assessment-attempts
docker-compose exec kafka kafka-topics --describe --topic assessment_attempts --zookeeper zookeeper:32181

In [ ]:
jupyter@python-20210907-215615:~/w205/project-2-dtrinidad002$ docker-compose exec kafka kafka-topics --describe --topic assessment_attempts --zookeeper zookeeper:32181
Topic: assessment_attempts      TopicId: y2dLSD4CRG2RlIwuEuLsrQ PartitionCount: 1       ReplicationFactor: 1    Configs: 
        Topic: assessment_attempts      Partition: 0    Leader: 1       Replicas: 1     Isr: 1

**Step 8: Check how many entries are there in the JSON file**  


In [ ]:
#command to count entries via mids container
docker-compose exec mids bash -c "cat /w205/project-2-dtrinidad002/assessment-attempts-20180128-121051-nested.json | jq '.[]' -c | wc -l"


In [ ]:
jupyter@python-20210907-215615:~/w205/project-2-dtrinidad002$ docker-compose exec mids bash -c "cat /w205/project-2-dtrinidad002/assessment-attempts-20180128-121051-nested.json | jq '.[]' -c | wc -l"

3280

There are ***3280*** entries in the nested json file

**Step 9: Publish event messages to Kafka topic**  


In [ ]:
# Use kafkacat to produce test messages to the "assessment-attempts" topic
docker-compose exec mids bash -c "cat /w205/project-2-dtrinidad002/assessment-attempts-20180128-121051-nested.json | jq '.[]' -c | kafkacat -P -b kafka:29092 -t assessment_attempts && echo 'Produced 3280 messages to topic assessment_attempts.'"

In [ ]:
jupyter@python-20210907-215615:~/w205/project-2-dtrinidad002$ docker-compose exec mids bash -c "cat /w205/project-2-dtrinidad002/assessment-attempts-20180128-121051-nested.json | jq '.[]' -c | kafkacat -P -b kafka:29092 -t assessment_attempts && echo 'Produced 3280 messages to topic assessment_attempts.'"

Produced 3280 messages to topic assessment_attempts.


**Step 10: Check messages sent to kafka**

In [ ]:
## code to print one message
docker-compose exec kafka kafka-console-consumer --bootstrap-server kafka:29092 --topic assessment_attempts --from-beginning --max-messages 1

**Step 11: Run Pyspark**  
- Spark will be used to consume messages that are published to Kafka

In [ ]:
# Run Pyspark
docker-compose exec spark pyspark

**Step 12:Consume messages from the "assessment_attempt" topic into Spak, then cache to suppress warning messages**

-json entries will turn into a spark data frame

In [ ]:
#code
spark.read.format("kafka").option("kafka.bootstrap.servers", "kafka:29092").option("subscribe","assessment_attempts").option("startingOffsets", "earliest").option("endingOffsets", "latest").load()

#cache
raw_assessments.cache()

In [ ]:
>>> raw_assessments = spark.read.format("kafka").option("kafka.bootstrap.servers", "kafka:29092").option("subscribe","assessment_attempts").option("startingOffsets", "earliest").option("endingOffsets", "latest").load()

>>> raw_assessments.cache()DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

**Step 13: Check the schema of "raw_assessments"**

-This raw data set is immutable, and can't be altered. 

In [ ]:
# Print Schema  
raw_assessments.printSchema()

In [ ]:
>>> raw_assessments.printSchema()

**Step 14: Convert the key and values into strings**
    
-In order to make the key and value readable, we need to convert them to strings. We would have to create a new data frame "assessments"
which will be a direct copy of the raw_assessments data frame, that we will need to mutate as we aim to make this dataset quireyable. 

In [ ]:
#Code to convert key and value as strings
assessments = raw_assessments.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

#Display assessments data frame
assessments.show()

#Double check the total number of messages have not changed
assessments.count()

In [ ]:
>>> assessments = raw_assessments.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
>>> assessments.show()


In [ ]:
>>> assessments.count()

In [ ]:
3280

**Step 15: Save the "assessment" df into hdfs as a parquet file**
- Parquet is optimized to work with complex data in bulk and features different ways for efficient data compression and encoding types.  This approach is best especially for those queries that need to read certain columns from a large table. 

In [ ]:
#Code
assessments.write.parquet("/tmp/assessments")

***(Open another terminal and verify the file in hdfs)***  
-You can see "assessments" was added to hadoop. 

In [ ]:
#code
docker-compose exec cloudera hadoop fs -ls /tmp/

In [ ]:
jupyter@python-20210907-215615:~/w205/project-2-dtrinidad002$ docker-compose exec cloudera hadoop fs 

-ls /tmp/
Found 3 items
drwxr-xr-x   - root   supergroup          0 2021-11-02 05:27 /tmp/assessments
drwxrwxrwt   - mapred mapred              0 2018-02-06 18:27 /tmp/hadoop-yarn
drwx-wx-wx   - root   supergroup          0 2021-11-02 04:06 /tmp/hive
jupyter@python-20210907-215615:~/w205/project-2-dtrinidad002$

**Step 16: Extract JSON fields**
- below you can see the first 3 rows

In [ ]:
from pyspark.sql import Row
import json

In [ ]:
#Create extracted_assessments df
extracted_assessments = assessments.rdd.map(lambda x: Row(**json.loads(x.value))).toDF()
extracted_assessments.show(3)

In [ ]:
>>> extracted_assessments.show(3)

+--------------------+-------------+--------------------+-----------------+--------------------+-----------------+------------+--------------------+--------------------+--------------------+
|        base_exam_id|certification|           exam_name|  keen_created_at|             keen_id|   keen_timestamp|max_attempts|           sequences|          started_at|        user_exam_id|
+--------------------+-------------+--------------------+-----------------+--------------------+-----------------+------------+--------------------+--------------------+--------------------+
|37f0a30a-7464-11e...|        false|Normal Forms and ...|1516717442.735266|5a6745820eb8ab000...|1516717442.735266|         1.0|Map(questions -> ...|2018-01-23T14:23:...|6d4089e4-bde5-4a2...|
|37f0a30a-7464-11e...|        false|Normal Forms and ...|1516717377.639827|5a674541ab6b0a000...|1516717377.639827|         1.0|Map(questions -> ...|2018-01-23T14:21:...|2fec1534-b41f-441...|
|4beeac16-bb83-4d5...|        false|The Principles of...|1516738973.653394|5a67999d3ed3e3000...|1516738973.653394|         1.0|Map(questions -> ...|2018-01-23T20:22:...|8edbc8a8-4d26-429...|
+--------------------+-------------+--------------------+-----------------+--------------------+-----------------+------------+--------------------+--------------------+--------------------+
only showing top 3 rows

In [ ]:
+--------------------+-------------+--------------------+-----------------+--------------------+-----------------+------------+--------------------+--------------------+--------------------+
|        base_exam_id|certification|           exam_name|  keen_created_at|             keen_id|   keen_timestamp|max_attempts|           sequences|          started_at|        user_exam_id|
+--------------------+-------------+--------------------+-----------------+--------------------+-----------------+------------+--------------------+--------------------+--------------------+
|37f0a30a-7464-11e...|        false|Normal Forms and ...|1516717442.735266|5a6745820eb8ab000...|1516717442.735266|         1.0|Map(questions -> ...|2018-01-23T14:23:...|6d4089e4-bde5-4a2...|
|37f0a30a-7464-11e...|        false|Normal Forms and ...|1516717377.639827|5a674541ab6b0a000...|1516717377.639827|         1.0|Map(questions -> ...|2018-01-23T14:21:...|2fec1534-b41f-441...|
|4beeac16-bb83-4d5...|        false|The Principles of...|1516738973.653394|5a67999d3ed3e3000...|1516738973.653394|         1.0|Map(questions -> ...|2018-01-23T20:22:...|8edbc8a8-4d26-429...|
+--------------------+-------------+--------------------+-----------------+--------------------+-----------------+------------+--------------------+--------------------+--------------------+
only showing top 3 rows

***(Check Schema of the "extracted_assessments")***

In [ ]:
#code
extracted_assessments.printSchema()

***(Can we expand the nested sequences further?)***

In [ ]:
#Code
sequences = spark.read.json(assessments.rdd.map(lambda x: x.value))
sequences.printSchema()

***NOTES:*** We can see that there is a lot of nested data in the sequences root key value, however, it's unlikely that it's useful for queriying. 


**Step 17: Make temp table in spark called "assessments"**

-We will use the extracted_assessments data frame  

In [ ]:
#Code
extracted_assessments.registerTempTable('assessments')

**Step 18: Run Spark SQL quiries against the temp table "assessments"**

In [ ]:
#Code
spark.sql("select user_exam_id, base_exam_id, exam_name, assessments.sequences.questions, max_attempts from assessments").show()

**Step 19: Save quirey to another partuet file in hdfs**  
-This data frame is a very basic and includes just the root columns (except the keen tables)  
-From Step 18, we can see that the questions sequence is still heavily nested


In [ ]:
#Code to created baseLevel_assessments data frame
baseLevel_assessments_info = spark.sql("SELECT user_exam_id, base_exam_id, exam_name, assessments.sequences.questions, max_attempts from assessments")

In [ ]:
#write the quirey results to parquet file in hadoop
baseLevel_assessments_info.write.parquet("/tmp/some_assessments_info")

**Step 20 check parquet file in a different terminal window**  
-You can see the "baseLevel_assessments_info" quirey was added 

In [ ]:
#Code
docker-compose exec cloudera hadoop fs -ls /tmp/

**Step 21: Extract questions field**  

-For this step we will use recursion 
-The Nested Key Values are 

In [17]:

def lambda_for_questions(x):
    raw_dict = json.loads(x.value)
    list = []
    count = 0
    for l in raw_dict["sequences"]["questions"]:
        count += 1
        dict = {"keen_id" : raw_dict["keen_id"], "count" : count, "id" : l["id"]}
        list.append(dict)
    return list

questions = assessments.rdd.flatMap(lambda_for_questions).toDF()

In [ ]:
questions.registerTempTable('questions')

In [ ]:
spark.sql("SELECT id, count FROM questions").show()

In [ ]:
#Join the "keen_id" column from the assessments df with the questions df
spark.sql("SELECT q.keen_id, a.keen_timestamp, q.id FROM assessments a JOIN questions q on a.keen_id = q.keen_id").show()

In [ ]:
#Display questions df with the added keen_id from the assessments df
questions.show()

**Step 23: Write quirey to parquet file** 

In [ ]:
questions.write.parquet("/tmp/questions")

#check temp files in hadoop on another terminal
docker-compose exec cloudera hadoop fs -ls /tmp

***Results***  
We have successfully saved 3 new parquet files in hadoop  
1.Assessments   
2.BaseLevel_assessments_info  
3.Questions

### Summary: 

We have successfully stepped through the process of demonstrating how to utilize docker clusters to consume and publish data streaming messages in Kafka. The JSON file contained user-assessment data which contained 10 root key variables and many nested key variables under the questions column. We also demonstrated the use of Spark to consume messages and transform them as needed. We then stored these messgages into Hadoop (HDFS). Though we would love to work with flat data tables, examples like this are likely what we will come across in the real world. Docker clusters like what's demonstrated here are what make the scale up possible. 

**Reccomendations**  
Without any clear documentation to describe the data attributes, I can only make the following assumptions of the data schema: 

1. The base_exam_id is linked to a specific exam_name 
2. User_exam_id is unique and does not repeat within the data set
3. Started_at column is formatted as (year-month-day-Time)
4. Certification "False" means the individual did not pass

Reccomendation 1: 
-Read deeper into the data attributes to get further context to understand the data. It's likely that some of my assumptions could be incorrect, or there may be data attributes i may have overlooked. 

Reccomendation 2:   
-Document a standard best practice for our data pipeline. This includes naming conventions for topics and data frames. 
